# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import nltk
from sqlalchemy import create_engine
import os
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle

nltk.download('punkt')
nltk.download('wordnet')
print(os.getcwd())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/workspace/home


In [2]:
# load data from database
engine = create_engine('sqlite://///workspace/home/MessagesDB.db')
df = pd.read_sql("SELECT * FROM MessagesDB", engine)
X = df.message
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    clean_tokens = [lemmatizer.lemmatize(tok, pos='v').lower().strip() for tok in tokens]

    return clean_tokens

In [4]:
tokenize(X[20])

['i',
 'would',
 'like',
 'to',
 'know',
 'if',
 'one',
 'of',
 'the',
 'radio',
 'ginen',
 'journalist',
 'die',
 '?']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
for index, column in enumerate(y_test):
    print(column, classification_report(y_test[column], y_pred[:, index]))

related              precision    recall  f1-score   support

          0       0.64      0.35      0.46      1566
          1       0.82      0.94      0.88      4988

avg / total       0.78      0.80      0.78      6554

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5428
          1       0.83      0.40      0.54      1126

avg / total       0.88      0.88      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.73      0.86      0.79      3888
          1       0.73      0.54      0.62      2666

avg / total       0.73      0.73      0.72      6554

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
(y_pred == y_test).mean()

related                   0.798444
request                   0.882514
offer                     0.995270
aid_related               0.729783
medical_help              0.919744
medical_products          0.955905
search_and_rescue         0.974672
security                  0.979554
military                  0.968264
child_alone               1.000000
water                     0.951327
food                      0.920659
shelter                   0.932255
clothing                  0.987031
money                     0.980317
missing_people            0.989930
refugees                  0.969789
death                     0.958804
other_aid                 0.872902
infrastructure_related    0.934086
transport                 0.955905
buildings                 0.949802
electricity               0.979402
tools                     0.994965
hospitals                 0.987183
shops                     0.995117
aid_centers               0.987489
other_infrastructure      0.958499
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
#pipeline.get_params()

In [11]:
parameters = {
    'clf__estimator__verbose': [0, 1]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__verbose': [0, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
cv.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__verbose': [0, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
cv.best_params_

{'clf__estimator__verbose': 0}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)
for index, column in enumerate(y_test):
    print(column, classification_report(y_test[column], y_pred[:, index]))

related              precision    recall  f1-score   support

          0       0.64      0.36      0.46      1566
          1       0.82      0.94      0.88      4988

avg / total       0.78      0.80      0.78      6554

request              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5428
          1       0.82      0.37      0.51      1126

avg / total       0.87      0.88      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.73      0.88      0.80      3888
          1       0.75      0.53      0.62      2666

avg / total       0.74      0.74      0.73      6554

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
(y_pred == y_test).mean()

related                   0.798749
request                   0.877937
offer                     0.995575
aid_related               0.738023
medical_help              0.920507
medical_products          0.956515
search_and_rescue         0.974367
security                  0.979554
military                  0.968111
child_alone               1.000000
water                     0.951175
food                      0.925694
shelter                   0.932713
clothing                  0.987641
money                     0.979860
missing_people            0.989930
refugees                  0.969942
death                     0.959109
other_aid                 0.873207
infrastructure_related    0.933628
transport                 0.954989
buildings                 0.952090
electricity               0.979554
tools                     0.994965
hospitals                 0.987183
shops                     0.995117
aid_centers               0.987336
other_infrastructure      0.958193
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.